# 최상위 모델인 openai의 모델들이 '인지왜곡'을 이해하고 있는지 먼저 알아보려 함
- 최상위 모델이 못하면, 그 아래 모델들도 못함
- 최상위 모델을 시켜서 되는지를 먼저 볼 것 + 어떻게 해야 되는지를 파악해서 거기서 탐색을 시작 할 것


- 방법1
  - LLM이 인지왜곡 유무를 판단할 수 있는가?
    -



논문 흐름 재구성
- 선행연구에서 구한 데이터 설명 (어떤 자격을 가진 사람들이 만들고, 평가했는지)
- 데이터를 LLM에 넣어 인지왜곡을 찾게 함, 선행연구 labeler들의 관점과 비교, 휴면 평가 <- 데이터 자체의 신뢰성에 의구심이 가는 상황이므로, 이걸 먼저 확인해야 함
- LLM이 더 그럴 듯 하게 잘 맞추는 것 같은데?
  - 데이터 전체 제공할테니 의심되면 직접 보렴
    - 이런 이유 때문에 잘 되는 거 같아
      - 전문성 있는 심리학자들과, 학습된 labeler들은 문장부터 직접 찾아내야 해서, 컨디션에 따라 평가능력이 좌우됨
      - 심리학자들은 경험에 기반하여, 약한 인지왜곡은 무시할 수 있음 (이 상황에서 이정도 생각은 괜찮아)
  - prompt 코드도 간단하게 만들어서 제공해줄께, 직접 만들어보셈
  - 심리 상당에서 힌트를 찾아내는데 도움을 줄 수 있으므로, 다 찾아내는게 중요해




설명을 요구하는게 휴면 평가하기 쉬움

재현이 잘 되지 않음
(1~n)개 모델로 여러번 생성하여, 비교해보는 것도 좋을 듯 -> voting을 통한 필터링?
전체를 찾아달라고 하면 재현 되나?
재현 가능한지 함 봐야 함

- prompt 대로 json형식으로 결과를 받을 수 있는지 먼저 보고, 그 다음에 재현 되는지 보기


In [3]:
# !pip install python-dotenv
# !pip install openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00


##### 공통부분

In [4]:
# 라이브러리 로드
from google.colab import drive
drive.mount('/content/drive')  # 구글 드라이브를 마운트

from dotenv import load_dotenv
import os
import time
import gc
from openai import OpenAI
import pandas as pd
import json
import re

# Google 드라이브의 특정 경로에 있는 .env 파일 로드
env_path = '/content/drive/MyDrive/사적문서/.env'
load_dotenv(env_path)

# 환경 변수에서 API 키 가져오기
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# OpenAI 클라이언트 초기화 (예시)
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# 글로벌 함수 설정

# api함수
# def get_chatgpt_response(message):
#     # OpenAI API를 통해 ChatGPT에게 한국어로 자연스럽게 다듬어 달라고 요청
#     response = client.chat.completions.create(
#         model="gpt-4o",
#         # messages=[
#         #     {"role": "system", "content": "You are a patient receiving psychological counseling. who can speak english only."},
#         #     {"role": "user",
#         #      "content": f"다음에 제공되는 2 문장은 너가 쓴 일기인데, 2~3문장이 빠져 있어 {input_01} {input_02}, 주어진 문장을 그대로 변경 없이 포함해서, 비어있는 내용을 영어로 써줘"}
#         # ]
#         messages = [
#             f"{message}"
#         ]
#     )
#     return response.choices[0].message.content


def get_chatgpt_response(message):
    # OpenAI API를 통해 ChatGPT에게 한국어로 자연스럽게 다듬어 달라고 요청
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": message}
        ]
    )
    return response.choices[0].message.content

In [6]:
# 데이터 불러오기
annotated = pd.read_csv('/content/drive/MyDrive/Project_now/2024_아이펠톤/raw_data/Annotated_data.csv')
c2d2 = pd.read_csv('/content/drive/MyDrive/Project_now/2024_아이펠톤/raw_data/C2D2/C2D20911.csv')
meta = pd.read_csv('/content/drive/MyDrive/Project_now/2024_아이펠톤/raw_data/meta/meta0911.csv', encoding='latin1')

print(annotated.columns)
print("----------------")
print(c2d2.columns)
print("----------------")
print(meta.columns)

Index(['Id_Number', 'Patient Question', 'Distorted part',
       'Dominant Distortion', 'Secondary Distortion (Optional)'],
      dtype='object')
----------------
Index(['Num', 'Scenario', 'Thought', 'Label'], dtype='object')
----------------
Index(['persona', 'pattern', 'pattern_def', 'thought', 'scenario',
       'persona_in_scenario', 'thought_in_scenario'],
      dtype='object')


In [7]:
# 전처리
## c2d2, 가능하면 문법 교정, 시나리오+생각
## meta, 시나리오

##### openai API에 prompt를 던질때, 일정한 형식으로 출력을 return 할 수 있나 확인

In [8]:
# # 한문장만 해보기
# data = annotated.iloc[0]['Patient Question']

# message = f"""
# 아래 텍스트에 인지왜곡이 있는지 여부를 판단하여 Yes or no 답변을 주되, Yes일 경우 아래 json format으로 요구사항에 맞춰 답변을 줘

# - json format
# "span" : "인지왜곡이 있는 부분 (1~3개 문장)"
# "label" : "인지왜곡 유형"
# "cause" : "인지왜곡에 대한 이유"
# "score" : "인지왜곡이 강한 정도, 점수"

# - 요구사항
# 1. 인지왜곡 점수
# - 인지왜곡과 정상적인 생각의 경계선에 있는 상태가 0점
# - 전문가 상담이 필요한 상태가 50점
# - 혼자 해결 불가능하여 주변 도움이 필요하다고 판단되는 상태가 100점
# 2. 약하더라도 인지왜곡이 있는 모든 부분을 다 찾아줘
# 3. 위 요구 사항 외 답변은 주지마

# - 원문
# {data}
# """

# print(message)

In [9]:
# 한문장만 해보기
n = 14

data = annotated.iloc[n]['Patient Question']

message = f'''
Determine whether there is cognitive distortion in the following text and provide a Yes or No answer. If the answer is Yes, respond according to the requirements below using the provided JSON format to list.

- JSON format:
[
  {{
    "span": "The part(s) containing cognitive distortion (1-3 sentences)",
    "label": "Type of cognitive distortion",
    "cause": "Reason for the cognitive distortion",
    "score": "Degree of cognitive distortion, score"
  }}
]

- Requirements:
1. Cognitive Distortion Score:
  - A borderline state between cognitive distortion and normal thinking is 0 points.
  - A state requiring professional consultation is 50 points.
  - A state judged as needing external help as it cannot be solved alone is 100 points.
2. Identify every part that shows even mild cognitive distortion.
3. Do not provide any response other than the above requirements.

- The following are the types of labels that can be used:
  - All-or-nothing thinking
  - Emotional Reasoning
  - Fortune-telling
  - Labeling
  - Magnification
  - Mental filter
  - Mind Reading
  - Overgeneralization
  - Personalization
  - Should statements

- following text
{data}
'''

print(message)


Determine whether there is cognitive distortion in the following text and provide a Yes or No answer. If the answer is Yes, respond according to the requirements below using the provided JSON format to list.

- JSON format:
[
  {
    "span": "The part(s) containing cognitive distortion (1-3 sentences)",
    "label": "Type of cognitive distortion",
    "cause": "Reason for the cognitive distortion",
    "score": "Degree of cognitive distortion, score"
  }
]

- Requirements:
1. Cognitive Distortion Score:
  - A borderline state between cognitive distortion and normal thinking is 0 points.
  - A state requiring professional consultation is 50 points.
  - A state judged as needing external help as it cannot be solved alone is 100 points.
2. Identify every part that shows even mild cognitive distortion.
3. Do not provide any response other than the above requirements.

- The following are the types of labels that can be used:
  - All-or-nothing thinking
  - Emotional Reasoning
  - Fortune-

In [10]:
answer = get_chatgpt_response(message)
print(answer)

Yes

```json
[
  {
    "span": "I basically want an outsider‚s point of view on whether I should accept this as part of the man I fell in love with and married, reject it on the grounds of how I feel (although I have a hard time with the thought of leaving him over it), or if we should try to work out some sort of compromise.",
    "label": "All-or-nothing thinking",
    "cause": "The speaker is considering only two extreme options: accepting the husband's behavior entirely or rejecting it, without considering more nuanced solutions.",
    "score": "50"
  },
  {
    "span": "When I confront him about it, he says he has a public sex fetish and needs to fulfill it, believing that it helps keep him ‚faithful‚ to me since I am no longer interested.",
    "label": "Emotional Reasoning",
    "cause": "The husband is using his emotional need for a public sex fetish to justify that it helps keep him 'faithful' without fully considering the impact on the relationship.",
    "score": "50"
  },
 

In [18]:
answer.split("\n", 2)[2].strip()

'```json\n[\n  {\n    "span": "I basically want an outsider‚s point of view on whether I should accept this as part of the man I fell in love with and married, reject it on the grounds of how I feel (although I have a hard time with the thought of leaving him over it), or if we should try to work out some sort of compromise.",\n    "label": "All-or-nothing thinking",\n    "cause": "The speaker is considering only two extreme options: accepting the husband\'s behavior entirely or rejecting it, without considering more nuanced solutions.",\n    "score": "50"\n  },\n  {\n    "span": "When I confront him about it, he says he has a public sex fetish and needs to fulfill it, believing that it helps keep him ‚faithful‚ to me since I am no longer interested.",\n    "label": "Emotional Reasoning",\n    "cause": "The husband is using his emotional need for a public sex fetish to justify that it helps keep him \'faithful\' without fully considering the impact on the relationship.",\n    "score": 

In [11]:
# 첫 줄의 YES 부분을 따로 변수에 담기
lable_yn = answer.split("\n")[0].strip()

# answer에서 'json' 텍스트 삭제
json_data = json.loads(answer.split("\n", 2)[2].strip())

# answer = answer.strip().replace("json","").replace("```","")

# # JSON 문자열을 파이썬 객체로 변환
# data = json.loads(answer)

# 데이터프레임 생성
df = pd.DataFrame(json_data)

# 결과 출력
print(lable_yn)
print("----------------")
print(df)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

##### 여러 데이터를 여러번 물어봐서, 나오는 답변을 휴먼이 비교하기